In [1]:
import pandas as pd
import os
import numpy as np

#os - operating system dependent. to read/write files
#pd - pandas for data analysis/modeling

data_path = 'data/'
dataframes_path = 'dataframes/'

#get DataFrames
#the training set. Daily historical data from January 2013 to October 2015
sales    = pd.read_csv(os.path.join(data_path, 'sales_train.csv.gz'))
#supplemental information about the items/products
items           = pd.read_csv(os.path.join(data_path, 'items.csv'),encoding ='ISO-8859-1')
#supplemental information about the items categories.
item_categories = pd.read_csv(os.path.join(data_path, 'item_categories.csv'),encoding ='ISO-8859-1')
#supplemental information about the shops.
shops           = pd.read_csv(os.path.join(data_path, 'shops.csv'),encoding ='ISO-8859-1')
#the test set. You need to forecast the sales for these shops and products for November 2015.
test            = pd.read_csv(os.path.join(data_path, 'test.csv.gz'))

#Return a tuple representing the dimensionality of the DataFrame
print(sales.shape)
print(test.shape)


(2935849, 6)
(214200, 3)


<h2>Sales Data</h2>
<p> Below are the findings from sales by analyzing following data.
<ul>
    <li>item_cnt_day has both negative and positive values.<p>We conclude that negative value represents item returns for that particular date/shop.</p></li>
    <li>item_price has both negative and positive values.<p>We concluded that negative value is either a noice/missing value. We ignored the fact that it could be some other representation and included in item_cnt_month since it appears only once.</p></li>
</ul>

</p>

In [8]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [9]:
sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [3]:
sales.loc[sales.item_price<0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,15.05.2013,4,32,2973,-1.0,1.0


In [9]:
sales.loc[sales.item_id==2973].loc[sales.shop_id==32]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
67427,29.01.2013,0,32,2973,2499.0,1.0
67428,25.01.2013,0,32,2973,2499.0,1.0
67429,22.01.2013,0,32,2973,2499.0,1.0
67430,21.01.2013,0,32,2973,2499.0,1.0
67431,18.01.2013,0,32,2973,2499.0,1.0
67432,17.01.2013,0,32,2973,2499.0,1.0
67433,15.01.2013,0,32,2973,2499.0,3.0
187844,05.02.2013,1,32,2973,2499.0,1.0
187845,14.02.2013,1,32,2973,2499.0,1.0
484682,23.05.2013,4,32,2973,1249.0,1.0


In [6]:
temp=sales.copy().groupby(['date','item_id'])
temp.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


<h2>Items Data</h2>
<p> Below are the findings from Items by analyzing following data.
<ul>
    <li>Item names are in russian language, and hard to interprit. We ignored the fact that name could affect sales.</p></li>
    <li>There are 22170 different type of items each belongs to a one item category group</li>
</ul>

</p>

In [10]:
items.head()

,item_name,item_id,item_category_id
0,! ÐÐ ÐÐÐÐ¡Ð¢Ð ÐÐÐÐÐÐÐÐÐÐ¯ (ÐÐ...,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***Ð ÐÐ£Ð§ÐÐ¥ Ð¡ÐÐÐÐ« (UNV) ...,2,40
3,***ÐÐÐÐ£ÐÐÐ¯ ÐÐÐÐÐ (Univ) ...,3,40
4,***ÐÐÐ ÐÐÐÐ (Ð¡Ð¢ÐÐÐÐ) ...,4,40


In [11]:
items.describe()

,item_id,item_category_id
count,22170.00000,22170.000000
mean,11084.50000,46.290753
std,6400.07207,15.941486
min,0.00000,0.000000
25%,5542.25000,37.000000
50%,11084.50000,40.000000
75%,16626.75000,58.000000
max,22169.00000,83.000000


In [30]:
len(items['item_id'].unique())


22170

<h2>Item Category Data</h2>
<p> Below are the findings from Items Categories by analyzing following data.
<ul>
    <li>Item Category names are in russian language, and hard to interprit.</li>
    <li>There are 84 different types of item categories</li>
</ul>

</p>

In [12]:
item_categories.head()

,item_category_name,item_category_id
0,PC - ÐÐ°ÑÐ½Ð¸ÑÑÑÑ/ÐÐ°ÑÑÐ½Ð¸ÐºÐ¸,0
1,ÐÐºÑÐµÑÑÑÐ°ÑÑ - PS2,1
2,ÐÐºÑÐµÑÑÑÐ°ÑÑ - PS3,2
3,ÐÐºÑÐµÑÑÑÐ°ÑÑ - PS4,3
4,ÐÐºÑÐµÑÑÑÐ°ÑÑ - PSP,4


In [13]:
item_categories.describe()

,item_category_id
count,84.000000
mean,41.500000
std,24.392622
min,0.000000
25%,20.750000
50%,41.500000
75%,62.250000
max,83.000000


In [31]:
len(item_categories['item_category_id'].unique())

84

<h2>Shops Data</h2>
<p> Below are the findings from Shops by analyzing following data.
<ul>
    <li>Shop names are in russian language, and hard to interprit.</li>
    <li>There are 60 different shops</li>
</ul>

</p>

In [14]:
shops.head()

,shop_name,shop_id
0,"!Ð¯ÐºÑÑÑÐº ÐÑÐ´Ð¶Ð¾Ð½Ð¸ÐºÐ¸Ð´Ð·Ðµ, 56 ÑÑ...",0
1,"!Ð¯ÐºÑÑÑÐº Ð¢Ð¦ ""Ð¦ÐµÐ½ÑÑÐ°Ð»ÑÐ½ÑÐ¹"" Ñ...",1
2,"ÐÐ´ÑÐ³ÐµÑ Ð¢Ð¦ ""ÐÐµÐ³Ð°""",2
3,"ÐÐ°Ð»Ð°ÑÐ¸Ñ Ð° Ð¢Ð Ð ""ÐÐºÑÑÐ±ÑÑ-ÐÐ¸Ð½...",3
4,"ÐÐ¾Ð»Ð¶ÑÐºÐ¸Ð¹ Ð¢Ð¦ ""ÐÐ¾Ð»Ð³Ð° ÐÐ¾Ð»Ð»""",4


In [15]:
shops.describe()

,shop_id
count,60.000000
mean,29.500000
std,17.464249
min,0.000000
25%,14.750000
50%,29.500000
75%,44.250000
max,59.000000


In [32]:
len(shops['shop_id'].unique())

60

<h2>Test Data</h2>
<p> Below are the findings from Test dataset
<ul>
    <li>Some shops are not presented in test data set (34th month)<p> There is no consistency for missing shops. for 34 th month following shops are missing {0, 1, 8, 9, 11, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54}. </p></li>
    <li>There are 84 different types of item categories</li>
</ul>

</p>

In [7]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [16]:
test.describe()

,ID,shop_id,item_id
count,214200.000000,214200.000000,214200.000000
mean,107099.500000,31.642857,11019.398627
std,61834.358168,17.561933,6252.644590
min,0.000000,2.000000,30.000000
25%,53549.750000,16.000000,5381.500000
50%,107099.500000,34.500000,11203.000000
75%,160649.250000,47.000000,16071.500000
max,214199.000000,59.000000,22167.000000


In [24]:
#Missing shops for test set (34th month)
set(range(0,60))-set(test['shop_id'].unique())

{0, 1, 8, 9, 11, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54}

In [23]:
#Missing number of shops per each month
for i in range(0,34):
    print(len(set(range(0,60))-set(sales.loc[sales.date_block_num==i,'shop_id'].unique())),end=" ")

15 14 14 14 15 14 14 15 15 14 15 14 14 14 12 11 11 11 10 9 10 8 10 10 10 13 14 13 16 17 17 18 17 16 